## Setting Up:

In [4]:
import sys
import os
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import zipfile
import cdsapi
import zipfile
import numpy as np
import glob
import xarray as xr

regions_path = '/home/mburlet/MBM/MassBalanceMachine/regions/French_Alps'
sys.path.insert(0, regions_path)
from scripts.config_FR import *

# Print paths to verify
print("Python path:", sys.path)
print("\nChecking if scripts directory exists:")
print(os.path.exists(os.path.join(regions_path, 'scripts')))
print("\nListing contents of regions directory:")
print(os.listdir(regions_path))

# Debug prints
print(f"1. Current working directory: {os.getcwd()}")
print(f"2. Regions path exists: {os.path.exists(regions_path)}")
print(f"3. Scripts directory exists: {os.path.exists(os.path.join(regions_path, 'scripts'))}")
print(f"4. Config file exists: {os.path.exists(os.path.join(regions_path, 'scripts', 'config_FR.py'))}")
print(f"5. sys.path: {sys.path}")

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

Python path: ['/home/mburlet/MBM/MassBalanceMachine/regions/French_Alps', '/home/mburlet/MBM/MassBalanceMachine/regions/French_Alps', '/home/mburlet/anaconda3/envs/MassBalanceMachine/lib/python311.zip', '/home/mburlet/anaconda3/envs/MassBalanceMachine/lib/python3.11', '/home/mburlet/anaconda3/envs/MassBalanceMachine/lib/python3.11/lib-dynload', '', '/home/mburlet/anaconda3/envs/MassBalanceMachine/lib/python3.11/site-packages', '/home/mburlet/MBM/MassBalanceMachine']

Checking if scripts directory exists:
True

Listing contents of regions directory:
['models', '1.3. ERA5Land-prepro.ipynb', '3.4. Feature selection.ipynb', '3.2. Train-ML-model.ipynb', '1.1. GLAMOS-prepro.ipynb', 'prcp_fac_array.npy', '3.5. Train with GeoMB.ipynb', '1.2. OGGM-datapulling.ipynb', '1.1. GLACIOCLIM-prepro.ipynb', '3.3. Spatial-analysis.ipynb', 'scripts', 'w_prcp_array.npy']
1. Current working directory: /home/mburlet/MBM/MassBalanceMachine/regions/French_Alps
2. Regions path exists: True
3. Scripts directory 

## Download monthly ERA5-Land variables:

In [5]:
RUN = True
if RUN:
    os.makedirs(path_ERA5_raw, exist_ok=True)
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-land-monthly-means', {
            'product_type': ['monthly_averaged_reanalysis'],
            'variable': [
                '10m_u_component_of_wind',
                '10m_v_component_of_wind',
                '2m_temperature',
                'forecast_albedo',
                'snow_cover',
                'snow_density',
                'snow_depth_water_equivalent',
                'snowfall',
                'snowmelt',
                'surface_latent_heat_flux',
                'surface_net_thermal_radiation',
                'surface_sensible_heat_flux',
                'surface_solar_radiation_downwards',
                'total_precipitation',
            ],
            'year': [
                '1950',
                '1951',
                '1952',
                '1953',
                '1954',
                '1955',
                '1956',
                '1957',
                '1958',
                '1959',
                '1960',
                '1961',
                '1962',
                '1963',
                '1964',
                '1965',
                '1966',
                '1967',
                '1968',
                '1969',
                '1970',
                '1971',
                '1972',
                '1973',
                '1974',
                '1975',
                '1976',
                '1977',
                '1978',
                '1979',
                '1980',
                '1981',
                '1982',
                '1983',
                '1984',
                '1985',
                '1986',
                '1987',
                '1988',
                '1989',
                '1990',
                '1991',
                '1992',
                '1993',
                '1994',
                '1995',
                '1996',
                '1997',
                '1998',
                '1999',
                '2000',
                '2001',
                '2002',
                '2003',
                '2004',
                '2005',
                '2006',
                '2007',
                '2008',
                '2009',
                '2010',
                '2011',
                '2012',
                '2013',
                '2014',
                '2015',
                '2016',
                '2017',
                '2018',
                '2019',
                '2020',
                '2021',
                '2022',
                '2023',
                '2024',
            ],
            'month': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
            ],
            'time': ['00:00'],
            "data_format": "netcdf",
            "download_format": "zip",
            'area': [
                47,
                3,
                43,
                8,
            ],
        }, path_ERA5_raw+'download.netcdf.zip')
    with zipfile.ZipFile(path_ERA5_raw+'download.netcdf.zip', 'r') as zip:
        zip.extractall(path_ERA5_raw)
    c.retrieve("reanalysis-era5-single-levels", {
            "product_type": ["reanalysis"],
            "variable": ["geopotential"],
            "year": ["2024"],
            "month": ["06"],
            "day": ["01"],
            "time": ["12:00"],
            "data_format": "netcdf"
        }, path_ERA5_raw+'era5_geopotential_pressure.nc')

2025-05-08 16:12:50,581 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-08 16:12:50,582 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-08 16:12:50,833 INFO Request ID is 171bd883-f4b7-47d0-948f-04739e718e02
2025-05-08 16:12:50,894 INFO status has been updated to accepted
2025-05-08 16:12:59,275 INFO status has been updated to running
2025-05-08 16:45:16,527 INFO status has been updated to successful


a35e89d1ff3587868e0a119932c5f79d.zip:   0%|          | 0.00/48.2M [00:00<?, ?B/s]

2025-05-08 16:45:21,991 INFO Request ID is ca77bc22-e0e0-496c-ac42-582050572e70
2025-05-08 16:45:22,100 INFO status has been updated to accepted
2025-05-08 16:45:30,465 INFO status has been updated to running
2025-05-08 16:45:35,732 INFO status has been updated to successful


aa9d83a880f1bb02cf32b143dfe1d3fa.nc:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

In [ ]:
"""
import cdsapi

dataset = "reanalysis-era5-land-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": [
        "2m_temperature",
        "snow_cover",
        "snow_density",
        "snow_depth_water_equivalent",
        "snowfall",
        "snowmelt",
        "forecast_albedo",
        "surface_latent_heat_flux",
        "surface_net_thermal_radiation",
        "surface_sensible_heat_flux",
        "surface_solar_radiation_downwards",
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "total_precipitation"
    ],
    "year": [
        "1957", "1958", "1959",
        "1960", "1961", "1962",
        "1963", "1964", "1965",
        "1966", "1967", "1968",
        "1969", "1970", "1971",
        "1972", "1973", "1974",
        "1975", "1976", "1977",
        "1978", "1979", "1980",
        "1981", "1982", "1983",
        "1984", "1985", "1986",
        "1987", "1988", "1989",
        "1990", "1991", "1992",
        "1993", "1994", "1995",
        "1996", "1997", "1998",
        "1999", "2000", "2001",
        "2002", "2003", "2004",
        "2005", "2006", "2007",
        "2008", "2009", "2010",
        "2011", "2012", "2013",
        "2014", "2015", "2016",
        "2017", "2018", "2019",
        "2020", "2021", "2022"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "zip",
    "area": [47, 3, 43, 8]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": ["geopotential"],
    "year": ["2024"],
    "month": ["06"],
    "day": ["01"],
    "time": ["12:00"],
    "data_format": "netcdf"
}

client = cdsapi.Client()
client.retrieve(dataset, request, "era5_geopotential_pressure.nc")
"""

HTTPError: 404 Client Error: Not Found for url: https://cds.climate.copernicus.eu/api/v2/retrieve/v1/processes/reanalysis-era5-land-monthly-means


In [10]:
! ls '/home/mburlet/scratch/data/DATA_MB/ERA5Land/raw/'

data_stream-moda.nc		     era5_geopotential_pressure_scratch.nc
download.netcdf.zip		     era5_monthly_averaged_data.nc
era5_geopotential_pressure_fixed.nc  era5_monthly_averaged_data_scratch.nc
era5_geopotential_pressure.nc


In [29]:
xr.open_dataset(path_ERA5_raw+'data_stream-moda.nc')

<xarray.Dataset> Size: 105MB
Dimensions:     (valid_time: 900, latitude: 41, longitude: 51)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 7kB 1950-01-01 ... 2024-12-01
  * latitude    (latitude) float64 328B 47.0 46.9 46.8 46.7 ... 43.2 43.1 43.0
  * longitude   (longitude) float64 408B 3.0 3.1 3.2 3.3 3.4 ... 7.7 7.8 7.9 8.0
    expver      (valid_time) <U4 14kB ...
Data variables: (12/14)
    u10         (valid_time, latitude, longitude) float32 8MB ...
    v10         (valid_time, latitude, longitude) float32 8MB ...
    t2m         (valid_time, latitude, longitude) float32 8MB ...
    fal         (valid_time, latitude, longitude) float32 8MB ...
    snowc       (valid_time, latitude, longitude) float32 8MB ...
    rsn         (valid_time, latitude, longitude) float32 8MB ...
    ...          ...
    smlt        (valid_time, latitude, longitude) float32 8MB ...
    slhf        (valid_time, latitude, longitude) float32 8MB ...
    str         (valid_time, latitude, longitude) float32 8MB ...
    sshf        (valid_time, latitude, longitude) float32 8MB ...
    ssrd        (valid_time, latitude, longitude) float32 8MB ...
    tp          (valid_time, latitude, longitude) float32 8MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-04-10T08:16 GRIB to CDM+CF via cfgrib-0.9.1...

In [14]:
def _handle_expver_dimension(ds_climate):
    """
    Handle dataset with or without expver dimension.
    Returns properly structured climate dataset.
    """
    if "number" not in ds_climate.coords:
        # Old format: reduce the expver dimension by summing
        print("Found expver dimension, reducing by sum...")
        return ds_climate.reduce(np.nansum, "expver")
    elif "number" in ds_climate.coords:
        # New format: expver exists as a coordinate only
        ds_climate = ds_climate.rename({'valid_time': 'time'}) # Coordinates have changed recently in the API, this is to keep compatibility with our code
        print("Found expver as coordinate but not dimension, removing unnecessary coordinate...")
        # Drop the expver coordinate as it's not needed and might cause issues
        return ds_climate
        #return ds_climate.drop_vars(["expver", "number"])
    else:
        # No expver at all - dataset is already in the expected format
        print("No expver found in dataset, proceeding with original structure...")
        return ds_climate


dc=xr.open_dataset(path_ERA5_raw+'data_stream-moda.nc')
dp=xr.open_dataset(path_ERA5_raw+'era5_geopotential_pressure.nc')
dc2 = _handle_expver_dimension(dc)
dp2 = _handle_expver_dimension(dp)

dc2.to_netcdf(path_ERA5_raw + "era5_monthly_averaged_data.nc")
dp2.to_netcdf(path_ERA5_raw + "era5_geopotential_pressure_fixed.nc")

Found expver as coordinate but not dimension, removing unnecessary coordinate...
Found expver as coordinate but not dimension, removing unnecessary coordinate...


In [15]:
display(xr.open_dataset(path_ERA5_raw+'era5_monthly_averaged_data.nc'))

display(xr.open_dataset(path_ERA5_raw+'era5_geopotential_pressure_fixed.nc'))

<xarray.Dataset> Size: 105MB
Dimensions:    (time: 900, latitude: 41, longitude: 51)
Coordinates:
    number     int64 8B ...
  * time       (time) datetime64[ns] 7kB 1950-01-01 1950-02-01 ... 2024-12-01
  * latitude   (latitude) float64 328B 47.0 46.9 46.8 46.7 ... 43.2 43.1 43.0
  * longitude  (longitude) float64 408B 3.0 3.1 3.2 3.3 3.4 ... 7.7 7.8 7.9 8.0
    expver     (time) <U4 14kB ...
Data variables: (12/14)
    u10        (time, latitude, longitude) float32 8MB ...
    v10        (time, latitude, longitude) float32 8MB ...
    t2m        (time, latitude, longitude) float32 8MB ...
    fal        (time, latitude, longitude) float32 8MB ...
    snowc      (time, latitude, longitude) float32 8MB ...
    rsn        (time, latitude, longitude) float32 8MB ...
    ...         ...
    smlt       (time, latitude, longitude) float32 8MB ...
    slhf       (time, latitude, longitude) float32 8MB ...
    str        (time, latitude, longitude) float32 8MB ...
    sshf       (time, latitude, longitude) float32 8MB ...
    ssrd       (time, latitude, longitude) float32 8MB ...
    tp         (time, latitude, longitude) float32 8MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-08T16:44 GRIB to CDM+CF via cfgrib-0.9.1...

<xarray.Dataset> Size: 4MB
Dimensions:    (time: 1, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * time       (time) datetime64[ns] 8B 2024-06-01T12:00:00
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     <U4 16B ...
Data variables:
    z          (time, latitude, longitude) float32 4MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T19:59 GRIB to CDM+CF via cfgrib-0.9.1...